In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# standard python
import numpy as np
import scipy
from scipy import stats
#import pathlib

import os
# plotting, especially for jupyter notebooks
import matplotlib
#matplotlib.rcParams['text.usetex'] = True # breaks for some endpoint labels
from matplotlib import pyplot as plt
from IPython.display import Image
import pandas as pd
# tensorflow
import tensorflow as tf
#tf.enable_eager_execution() # needed for tf version 1 or it stages operations but does not do them
from tensorflow import keras
from keras import layers, models, optimizers, losses, metrics
from tensorflow.keras import layers, regularizers
tf.keras.backend.clear_session()  # For easy reset of notebook state.
# local routines
#from chemdataprep import load_PDBs,load_countsfromPDB,load_diametersfromPDB,find_chemnames
#from toxmathandler import load_tscores

#checkpoint_path = "/home2/ajgreen4/Read-Across_w_GAN/Models/cp.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

print("tensorflow version",tf.__version__,". Executing eagerly?",tf.executing_eagerly())
print("Number of GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import weightedviews
#import toxmathandler
import qm9datapreparation
test

In [ ]:
from qm9datapreparation import all_molecules

In [ ]:
print(len(all_molecules))
print(all_molecules[40000])
len(all_molecules)


In [ ]:
unique_atoms = set()

for molecule in all_molecules:
    for atom in molecule:
        unique_atoms.add(atom[0])
unique_atoms = sorted(unique_atoms)
print("atom types:", unique_atoms)

In [ ]:
from weightedviews import load_qm9_data
from weightedviews import speciesmap
ws, vs, Natoms, Nviews = load_qm9_data(all_molecules, speciesmap, setNatoms= None, setNviews= None, carbonbased= False, verbose= 1)

In [ ]:
from ese import elements

In [ ]:
ese_element = np.array(elements)
ese_element.shape

In [ ]:
from sklearn.model_selection import train_test_split

all_molecules_train, all_molecules_test, ese_element_train, ese_element_test = train_test_split(all_molecules, ese_element, test_size= 0.2)

In [ ]:
print(len(all_molecules_test))
print(len(all_molecules_train))
print(len(ese_element_test))
print(len(ese_element_train))

In [ ]:
qm9_train = all_molecules_train
qm9_test = all_molecules_test
qm9_labels_train = ese_element_train
qm9_labels_test= ese_element_test

In [ ]:
ws_qm9_train, vs_qm9_train, Natoms_train, Nviews_train = load_qm9_data(qm9_train, speciesmap, setNatoms= None, setNviews= 29, carbonbased= False, verbose= 1)
qm9G_train = [ws_qm9_train, vs_qm9_train]
ws_qm9_test, vs_qm9_test, Natoms_test, Nviews_test = load_qm9_data(qm9_test, speciesmap, setNatoms= None, setNviews= 29, carbonbased= False, verbose= 1)
qm9G_test = [ws_qm9_test, vs_qm9_test]

In [ ]:
labelsG_train = qm9_labels_train
labelsG_test = qm9_labels_test
Ntoxicity =1
ws_train, vs_train = ws_qm9_train, vs_qm9_train
ws_test, vs_test = ws_qm9_test, vs_qm9_test
dataG_train = [ws_train, vs_train]
dataG_test = [ws_test, vs_test]

In [ ]:
print(type(labelsG_train))
print(type(labelsG_test))
print(type(dataG_train))
print(type(dataG_test))

In [ ]:
print(ws_train.shape)
print(vs_train.shape)
print(ws_test.shape)
print(vs_test.shape)
print(labelsG_train.shape)
print(labelsG_test.shape)



Neural Network code
Constructor routines

In [ ]:
# generic dense NN
def multiDense(Nin,Nout,Nhidden,widthhidden=None,kernel_regularizer=None):
    """Construct a basic NN with some dense layers.
    
    :parameter Nin: The number of inputs
    :type Nin: int
    :parameter Nout: The number of outputs
    :type Nout: int
    :parameter Nhidden: The number of hidden layers.
    :type Nhidden: int
    :parameter widthhidden: The width of each hidden layer.
        If left at None, Nin + Nout will be used.
    :parameter kernel_regularizer: the regularizer to use, such as regularizers.l2(0.001)
    :type kernel_regularizer: tensorflow.keras.regularizers.xxx
    :returns: The NN model
    :rtype: keras.Model
    
    """
    if widthhidden is None:
        widthhidden = Nin + Nout
    x = inputs = keras.Input(shape=(Nin,), name='multiDense_input')
    if kernel_regularizer is not None:
        print("Using regularization")
    for i in range(Nhidden):
        x = layers.Dense(widthhidden, activation='relu', kernel_regularizer=kernel_regularizer,name='dense'+str(i))(x)
#        x = layers.Dense(widthhidden, name='dense'+str(i))(x)
#        x = tf.nn.leaky_relu(x, alpha=0.05)
#    outputs = layers.Dense(Nout, activation='linear',name='multiDense_output')(x)
    outputs = layers.Dense(Nout,name='multiDense_output')(x)
    #outputs = tf.nn.leaky_relu(outputs, alpha=0.05)
    return keras.Model(inputs=inputs, outputs=outputs)#, name='multiDense')
if 1:
    # manual check of multiDense
    mmd = multiDense(10,4,3)
    mmd.summary()

In [ ]:
def parallelwrapper(Nparallel,basemodel,insteadmax=False):
    """Construct a model that applies a basemodel multiple times and take a weighted sum (or max) of the result.
    
    :parameter Nparallel: The number of times to apply in parallel
    :type Nparallel: int
    :parameter basemodel: a keras.Model inferred to have Nin inputs and Nout outputs.
    :type basemodel: a keras.Model
    :parameter insteadmax: If True, take the max of the results of the basemodel instead of the weighted sum.
        For compatibility, the model is still constructed with weights as inputs, but it ignores them.
    :type insteadmax: Boolean
    :returns: model with inputs shape [(?,Nparallel),(?,Nin,Nparallel)] and outputs shape (?,Nout).
        The first input is the scalar weights in the sum.
    :rtype: keras.Model
    
    Note: We could do a max over the parallel applications instead of or in addition to the weighted sum.
    
    """
    # infer shape of basemodel inputs and outputs
    Nin =  basemodel.inputs[0].shape[1]
    Nout =  basemodel.outputs[0].shape[1]
    
    # Apply basemodel Nparallel times in parallel
    # create main input (?,Nparallel,Nin) 
    parallel_inputs = keras.Input(shape=(Nparallel,Nin), name='parallelwrapper_input0')
    # apply base NN to each parallel slice; outputs (?,Nparallel,Nout)
    if False:
        # original version, stopped working at some tensorflow update
        xb = basemodel(parallel_inputs) # worked in earlier tensorflow
        #xb = tf.map_fn(basemodel,parallel_inputs) # another version that fails
    else:
        # newer version, works but makes summary and graphing cumbersome
        # unstack in the Nparallel directio
        parallel_inputsunstacked = tf.keras.ops.unstack(parallel_inputs, Nparallel, 1)
        # apply base NN to each 
        xbunstacked = [basemodel(x) for x in parallel_inputsunstacked]
        # re-stack
        xb = tf.keras.ops.stack(xbunstacked,axis=1)
    
    # create input scalars for weighted sun (?,Nparallel)
    weight_inputs = keras.Input(shape=(Nparallel,), name='parallelScalars')
    if insteadmax:
        # take max over the Nparallel direction to get (?,1,Nout)
        out = layers.MaxPool1D(pool_size=Nparallel)(xb)
        # reshape to (?,Nout)
        out = layers.Reshape((Nout,))(out)
    else:
        # do a weighted sum over the Nparallel direction to get (?,Nout)
        out = layers.Dot((-2,-1))([xb,weight_inputs])
    
    return keras.Model(inputs=[weight_inputs,parallel_inputs], outputs=out, name='parallelwrapper')
if 1:
    # manual check
    mmd = multiDense(10,4,3)
    mmd.summary()
    mpw = parallelwrapper(5,mmd,insteadmax=0)
    mpw.summary()

In [ ]:
def init_generator(data,labels,baselayers,Nfeatures,endlayers,base_regularizer=None,end_regularizer=None):
    """Initialize the generator neural net.
    
    :returns: return generator and descrimina NN.
    :rtype: keras.Model
    
    """
    ## Option changing how results of each view are aggregated
    insteadmax = False # Does weighted average; original design
    #insteadmax = True # Does max instead of weighted average (for both G and D)

    # G
    # base NN
    Gbase = multiDense(data[1].shape[2],Nfeatures,baselayers,kernel_regularizer=base_regularizer) 
    # parallel view wrapper
    Gpw = parallelwrapper(Nviews,Gbase,insteadmax)
    # features to toxicity
    #Gft = multiDense(Nfeatures,labels.shape[1],endlayers,kernel_regularizer=end_regularizer)
    #we got an error when running original line since energies are one dimnsional array so we can interpret the single dimension as having only one dimension
    Gft = multiDense(Nfeatures,1,endlayers,kernel_regularizer=end_regularizer) 
    # string together
    generator = keras.Model(inputs=Gpw.inputs,outputs=Gft(Gpw.outputs),name='generator')
    # make trainable
    generator.compile(optimizer='adam',loss='mse')
    #generator.summary()
    # previously did better with Nfeatures=Ntoxicity and no Gft

    if 0:
        # sanity checks that model is working
        print("Sanity check:")
        ws,vs = data
        gbv0call = Gbase(vs[:,0,:]).numpy()
        gbv0predict = Gbase.predict(vs[:,0,:])
        print("base: 0 ?==", np.linalg.norm(gbv0call-gbv0predict))
        gpwcall = Gpw([ws,vs]).numpy()
        gpwpredict = G
        pw.predict([ws,vs])
        print("wrapper: 0 ?==",np.linalg.norm(gpwcall-gpwpredict))
        gencall = generator([ws,vs]).numpy()
        genpredict = generator.predict([ws,vs])
        print("whole: 0 ?==",np.linalg.norm(gencall-genpredict))
        
    return generator
baselayers = 2  # hidden layers before weighted sum
base_reg = 0.1  # regularization for the base layers
Nfeatures = 1  # number of outputs of weighted sum
endlayers = 1  # hidden layers after weighted sum
end_reg = 0.1  # regularization for the end layers

if base_reg == 0:
    base_regularizer = None
else:
    base_regularizer = regularizers.l2(base_reg)

if end_reg == 0:
    end_regularizer = None
else:
    end_regularizer = regularizers.l2(end_reg)  #???

print("(baselayers, base_reg, Nfeatures, endlayers, end_reg) =",
      (baselayers, base_reg, Nfeatures, endlayers, end_reg))
# compile model with options
generator = init_generator(dataG_train,labelsG_train,baselayers,Nfeatures,endlayers,
                           base_regularizer=base_regularizer,end_regularizer=end_regularizer)
generator.compile(optimizer='adam',loss='mse')
#generator.summary()
#change loss to MAE for energies




In [ ]:
%%time
# fit
BATCH_SIZE = 32
if 1: 
    history = generator.fit(dataG_train,labelsG_train,
                  epochs=200,batch_size=BATCH_SIZE,verbose=0,
                  validation_data=(dataG_test, labelsG_test))
    print("train loss=",generator.evaluate(dataG_train,labelsG_train,verbose=0))
    print("test loss=",generator.evaluate(dataG_test,labelsG_test,verbose=0))
#     generator.fit(dataG_train,labelsG_train,epochs=1000,batch_size=BATCH_SIZE,verbose=0)
#    generator.save(modelpath+"AG-model-RT.h5")

In [ ]:
# plot predictions of G versus truth
def PvT_plot(model,data,labels,legend=None,title=None,doresidual=False):
    """Construct a plot of the true labels (x-axis) vs the data generated by the model (y-axis).
    
    :parameter model: the model (e.g. NN)
    :type model: keras.model
    :parameter data: the data that can be input to the model
    :type data: numpy.array
    :parameter labels: the true outputs corresponding to the data
    :type labels: numpy.array
    :parameter legend: The string labels corresponding to the columns of labels 
    :type legend: None or list of str
    :parameter title: A title for the plot
    :type title: None or string
    :parameter doresidual: If true, plot the residual instead
    :type doresidual: Boolean
    """
        
    gen_lab = model.predict(data)
    if doresidual:
        gen_lab = gen_lab - labels

    plt.figure()
    ax = plt.subplot(111)        
    if legend is None:
        for i in range(labels.shape[1]):
            plt.plot(labels[:,i],gen_lab[:,i],'o')
    else:
        for i in range(labels.shape[1]):
            # include legend
            plt.plot(labels[:,i],gen_lab[:,i],'o', label=legend[i])
        ax.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
    ax.set_xlabel('True Values')
    if doresidual:
        ax.set_ylabel('Residual Values')
    else:
        ax.set_ylabel('Generated Values')
        # reference line
        mintrue = np.min(labels)
        maxtrue = np.max(labels)
        plt.plot([mintrue,maxtrue],[mintrue,maxtrue])
    if title is None:
        title = ''
    if 1:
        if not doresidual:
            gen_lab = gen_lab - labels
        MSE = np.square(gen_lab).mean()
        title = title+" Mean Squared Error="+str(MSE)
        print("Mean Squared Error: ", MSE)
    plt.title(title)
    plt.show()

In [ ]:
# Plot training and validation loss
if 1:
    plt.figure()
    if base_regularizer is None and end_regularizer is None:
        plt.title('Loss (no regularization)')
    else:
        plt.title(f'Loss, including regularization (base_reg,end_reg)=({base_reg},{end_reg})')
    plt.semilogy(history.history['loss'], label='train')
    plt.semilogy(history.history['val_loss'], label='test')
    plt.ylim((0, plt.ylim()[1]))  # Set lower ylim to 0
    plt.legend()
    plt.show()